In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [0]:
ruta_base_atributos = 'default.base_atributos'
ruta_base_trx = 'default.base_trx'
ruta_base_cliente = 'default.base_cliente'

In [0]:
print ('dev')

In [0]:
# Leer la tabla usando Spark
df_atributos_sp = spark.table(ruta_base_atributos)
df_trx_sp = spark.table(ruta_base_trx)
df_cliente_sp = spark.table(ruta_base_cliente)

In [0]:
# Si quieres pasarlo a Pandas
df_atributos = df_atributos_sp.toPandas()
df_trx = df_trx_sp.toPandas()
df_cliente = df_cliente_sp.toPandas()

In [0]:
df_cliente = df_cliente.drop(columns=['periodo'])

In [0]:
df = df_trx.merge(df_cliente, on=['id_cliente'], how='left')
df = df.merge(df_atributos, on=['periodo', 'id_cliente'], how="left")

In [0]:
df_trx.shape[0], df.shape[0]

In [0]:
df = df[(df['flg_churn']== 0) | (df['flg_churn']== 1)]

In [0]:
df = df[~df['tasa'].isnull()]
df = df[~df['periodo_creacion'].isnull()]
df = df[~df['tipo_producto'].isnull()]
df = df[~df['segmento_pago'].isnull()]
df = df[~df['canal'].isnull()]
df = df[~df['segmento_cliente'].isnull()]
df = df[~df['crossell'].isnull()]
df = df[~df['departamento'].isnull()]

In [0]:
df.isnull().sum()[df.isnull().sum()>0]

In [0]:
df['flg_churn'] = df['flg_churn'].astype(int)
df['segmento_cliente'] = df['segmento_cliente'].astype(int)

In [0]:
df.dtypes

In [0]:
# Convertir a Spark DataFrame
df_spark = spark.createDataFrame(df)

# Guardar en el catálogo
df_spark.write.mode("overwrite").saveAsTable("default.base")